In [1]:
pip install scipy


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install sympy


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from sympy import combinatorics
from sympy.combinatorics import Permutation, PermutationGroup
from sympy.combinatorics import Polyhedron
from sympy.functions.combinatorial.numbers import stirling, bell
from math import factorial
from itertools import permutations
from scipy.stats import chisquare
import numpy as np

In [4]:
def count_records_exp(k, n): 
    return stirling(n,k, kind=1) / factorial(n)

In [5]:
def gen_permutation_selection(n):
    return [Permutation(x) for x in permutations(range(n))]

In [6]:
def count_records(permutation: Permutation): 
    maximum = -1
    records = 0 
    for index, elem in enumerate(permutation.list()): 
         if elem > maximum: 
            records += 1
            maximum = elem    
    return records

In [34]:
def analyze_records(n): 
    selection = gen_permutation_selection(n)
    records_counts_ob, records_counts_exp = [0] * n, [0] * n
    
    length = factorial(n)
    for item in selection: 
        records_counts_ob[count_records(item) - 1] += 1
    for i in range(n): 
        records_counts_exp[i] = count_records_exp(i + 1, n) * length
        
    return np.array([float(i) for i in records_counts_ob]), np.array([float(i) for i in records_counts_exp])

In [35]:
(ob, exp) = process(5)

In [36]:
chisquare(f_obs=np.array(ob), f_exp=np.array(exp))

Power_divergenceResult(statistic=0.0, pvalue=1.0)